<a href="https://colab.research.google.com/github/dylanwalker/thoughts/blob/master/_notebooks/2022-01-02-HouseSearch2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orange County House Search

The following are some considerations for buying a house in Orange County, CA

In [13]:
#hide
#!pip install --upgrade -q plotly
#!pip install --upgrade -q pandas
#!pip install -q plotlyhtmlexporter

import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px

pd.options.plotting.backend = "plotly"

In [14]:
#hide
sheetId = '1az-gajkCWX4GwgJnNMFX8gfeRvhRxAY3QMTowj0YTwE'
hdf = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?format=csv',
                 skiprows=2)
hdf = hdf.iloc[:,1:] # skip first blank column
hdf.rename(columns={col:col.replace(' ','').lower() for col in hdf.columns},inplace=True)
for col in hdf.columns[4:]:
  if hdf[col].dtype=='O':
    hdf[col] = pd.to_numeric(hdf[col].str.replace('[^0-9.]',''))
hdf.delta_price = hdf.delta_price/100
hdf['home2'] = hdf.home.apply(lambda x: x.split(',')[0])
hdf['zip'] = hdf['home'].str.split(',').str[-1].str.split(' ').str[-1]
hdf.head()

,status,area,home,zillowlink,price,price2019,bed,bath,sqft,lot,psqft,psqft2019,delta_price,hoafee,dylanq,kateq,jayq,tinaq,maddyq,aveq,home2,zip
0,Coming Soon,Fullerton,"3101 Flintridge Dr, Fullerton, CA 92835",https://www.zillow.com/homedetails/3101-Flintr...,1675000,1300000,4,3,3183,28750,526,408,0.29,0,NaN,NaN,NaN,NaN,NaN,0.0,3101 Flintridge Dr,92835
1,Coming Soon,Ladera Ranch,"20 Winslow St, Ladera Ranch, CA 92694",https://www.zillow.com/homedetails/20-Winslow-...,1700000,1200000,5,4,3433,6322,495,350,0.42,200,NaN,NaN,NaN,NaN,NaN,0.0,20 Winslow St,92694
2,For Sale,Mission Viejo,"5 Christopher St, Mission Viejo, CA 92694",https://www.zillow.com/homes/for_sale/64703883...,2088000,1600000,4,5,3906,9315,535,410,0.31,497,NaN,NaN,NaN,NaN,NaN,0.0,5 Christopher St,92694
3,Coming Soon,San Juan Capistrano,"32162 Calle Los Elegantes, San Juan Capistrano...",https://www.zillow.com/homes/for_sale/25577243...,2199000,1500000,5,5,4473,11761,492,335,0.47,325,NaN,NaN,NaN,NaN,NaN,0.0,32162 Calle Los Elegantes,92675
4,Coming Soon,San Juan Capistrano,"32162 Calle Los Elegantes, San Juan Capistrano...",https://www.zillow.com/homedetails/32162-Calle...,2199000,1500000,5,5,4473,11761,492,335,0.47,325,NaN,NaN,NaN,NaN,NaN,0.0,32162 Calle Los Elegantes,92675


In [15]:
#hide
# Read in HPI data and convert to Pandas DataFrame
hpidf =  pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?gid=1182115701&format=csv',
                 skiprows=6,dtype='str')
hpidf = pd.DataFrame(hpidf.values[1:],columns=['zip','year','percent_change','hpi','hpi_1990','hpi_2000'])
for col in hpidf.columns[1:]:
  hpidf[col] = pd.to_numeric(hpidf[col].str.replace('[^0-9.]',''),errors='coerce')
hpidf.head()

,zip,year,percent_change,hpi,hpi_1990,hpi_2000
0,01001,1986,13.67,113.67,70.65,69.80
1,01001,1987,21.20,137.77,85.63,84.60
2,01001,1988,17.38,161.72,100.52,99.31
3,01001,1989,1.14,163.57,101.67,100.45
4,01001,1990,1.64,160.89,100.00,98.80


In [16]:
#hide
azdf = hdf.groupby(['area','zip']).count().reset_index()[['area','zip']]

In [17]:
#hide
hpiyeardf = azdf.merge(hpidf,left_on='zip',right_on='zip',how='inner')
hpiyeardf = hpiyeardf[hpiyeardf.year>2000].groupby(['area','year']).mean()['hpi_2000']#.reset_index()
hpiyeardf.head()

area          year
Coto De Caza  2001    111.32
              2002    122.11
              2003    134.06
              2004    156.78
              2005    193.51
Name: hpi_2000, dtype: float64

In [18]:
#hide
p21df = hdf.groupby('area').mean()['delta_price'].reset_index()
p21df = hpiyeardf.reset_index()[hpiyeardf.reset_index().year==2019].merge(p21df,left_on='area',right_on='area')
p21df['year']=2021
p21df.hpi_2000 = p21df.hpi_2000*(1+p21df.delta_price)
p21df = p21df[['area','year','hpi_2000']]

In [19]:
#hide
# add projection of 2021 based on selling prices of homes (this is an assumption as these houses may be overpriced and may not sell)
hpiyeardf = pd.concat([hpiyeardf,p21df.groupby(['area','year']).mean()['hpi_2000']])

## Historical Prices in the OC Areas

Below are shown the historical prices of houses in each area relative to their estimated price in  2000. The final points in 2021 are based on average asking price per area of the houses we have noted. 

In [20]:
#hide_input
# Make Historical house prices figure
trendline = px.scatter(hpiyeardf.reset_index().groupby('year').mean()['hpi_2000'].reset_index(),x='year',y='hpi_2000',trendline='ols').data[1].update(line_color='black')
fig = hpiyeardf.unstack(level=0).plot();


#fig = hpiyeardf.unstack(level=0).plot();
fig.add_traces(trendline)
fig.update_yaxes(title_text="HPI (% of 2000 baseline)")
fig.update_layout(title="Orange County Houses: Historical Prices since 2000")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: 2021 values are projected from selling prices over 2019 price")


In [21]:
#hide
#import plotly.io as pio
#pio.renderers.default = "notebook+pdf"

## Guide to find houses
A comparison of the rating vs asking price of houses are shown below, along with their current status and area. Click the corresponding unit in the legend to remove houses from the plot.  

In [22]:
#hide_input
# Rating vs. Price
filter = hdf.aveq>5 #suppress houses with lower than 5 rating
#colorProp = "area"
colorProp = "status"
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[7]})<br><br>%{customdata[0]}<br>$%{x:,} (%{customdata[3]:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
symbol_map = {"For Sale":"circle",
              "Coming Soon":"circle",
              "Off Market":"square",
              "Pending":"cross",
              "Under Contract":"cross",
              "Sold":"x"}

fig = px.scatter(hdf[filter],x="price",y="aveq",text="home2",size='sqft',color="area",symbol="status",symbol_map=symbol_map,hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","status"])
trendline = px.scatter(hdf[filter],x="price",y="aveq",trendline="ols",trendline_options=dict(log_x=True)).data[1]
fig.update_traces(hovertemplate=hovertemplate)
fig.add_traces(trendline)
fig.update_yaxes(title_text="Rating (average)")
fig.update_traces(textposition='top center',textfont_size=10)#,marker=dict(line=dict(width=hdf[filter].delta_price*10,color='DarkSlateGrey'))) #<-- this doesn't work!
fig.update_layout(title="Orange County Houses: Rating vs Price")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: point size = sqft")
fig.write_html('H21_rating_vs_price.html')
fig.show()

## Price Change since 2019
The median increase in asking price relative to 2019 is `40%`. Houses below this median are depicted in the green area of the plot, while houses above this are depicted in the red area of the plot.

In [23]:
#hide_input
#Rating vs. Price change
symbol_map = {"For Sale":"circle",
              "Coming Soon":"circle",
              "Off Market":"square",
              "Pending":"cross",
              "Under Contract":"cross",
              "Sold":"x"}

trend = px.scatter(hdf,y="aveq",x="delta_price",trendline="ols") # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="delta_price",color="area", size="sqft", symbol="status",symbol_map=symbol_map,text="home2",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath"])
hovertemplate = "<b>%{hovertext}</b><br><br>%{customdata[0]}<br>$%{customdata[3]:,} (%{x:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.update_traces(textposition='top center',textfont_size=8)
fig.add_trace(trend.data[1])
fig.update_layout(title="Orange County Houses: Rating vs Price change (since 2019)")
fig.update_xaxes(title_text="Price change (since 2019)",tickformat='.0%')
fig.update_yaxes(title_text="Rating (average)")
fig.add_vrect(x0=-0.1, x1=hdf.delta_price.mean(), line_width=0, fillcolor="green", opacity=0.1)
fig.add_vrect(x0=hdf.delta_price.mean(), x1=1.05, line_width=0, fillcolor="red", opacity=0.1)
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: green/red boundary is at the median price increase")
fig.write_html('H21_rating_vs_delta_price.html')
fig.show()

## Rating, Price and Square Footage

Our average house rating grows less than linearly with the increase in square footage, with any houses achieving an `average rating > 7` when they exceed `3000` square feet: 

In [24]:
#hide_input
# Rating vs Sqft

trend = px.scatter(hdf,y="aveq",x="sqft",trendline="ols",trendline_options=dict(log_x=True)).data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="sqft",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath"])
hovertemplate = "<b>%{hovertext}</b><br><br>%{customdata[0]}<br>$%{customdata[2]:,} (%{customdata[3]:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{x} sqft<br>%{customdata[1]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Rating vs Square Footage")
fig.update_xaxes(title_text="Square Feet")
fig.update_yaxes(title_text="Rating (average)")
fig.write_html('H21_rating_vs_sqft.html')
fig.show()

The following shows that we are ***mostly insensitive to price after the square footage of the house is accounted for*** (with `0.3` change in average rating per `100` increase in price/sqft):

In [25]:
#hide_input
# Rating vs Price per sqft

trend = px.scatter(hdf,y="aveq",x="psqft",trendline="ols").data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="psqft",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath"])
hovertemplate = "<b>%{hovertext}</b><br><br>%{customdata[0]}<br>$%{customdata[3]:,} (%{customdata[4]:.0%})<br>%{customdata[6]} bed, %{customdata[7]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[5]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Rating vs Price per square foot")
fig.update_xaxes(title_text="Price per Square Foot")
fig.update_yaxes(title_text="Rating (average)")
fig.write_html('H21_rating_vs_psqft.html')
fig.show()

In [26]:
#hide_input
# Sqft vs Price

trend = px.scatter(hdf,y="sqft",x="price",trendline="ols").data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="sqft",x="price",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","aveq"])
hovertemplate = "<b>%{hovertext}</b><br><br>%{customdata[0]}<br>$%{x:,} (%{customdata[2]:.0%})<br>%{customdata[4]} bed, %{customdata[5]} bath<br>%{y} sqft<br>%{customdata[1]:,} lot size<br>$%{customdata[3]} HOA<br>%{customdata[6]}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Sqft vs Price")
fig.update_xaxes(title_text="Price")
fig.update_yaxes(title_text="Square Feet")
fig.write_html('H21_sqft_vs_price.html')
fig.show()

In [27]:
#hide
# Mount google drive (so I can try to export whole notebook as html)

# from google.colab import drive
# drive.mount('/content/drive/')

In [28]:
#hide
# Convert the notebook to html

# %%shell
# cp '/content/drive/MyDrive/Colab Notebooks/HouseSearch2021.ipynb' ./
# jupyter nbconvert --to html /content/HouseSearch2021.ipynb

## House of interest

The following houses are sorted by the highest average rating, followed by the price:

In [29]:
#hide_input
# Filter only For Sale with high ratings

hdf.loc[(hdf.status=="For Sale")&(hdf.aveq>7),["home","zillowlink","price","sqft","aveq"]]

,home,zillowlink,price,sqft,aveq
8,"30752 Paseo Del Niguel, Laguna Niguel, CA 92677",https://www.zillow.com/homes/30752-Paseo-Del-N...,5900000,8200,9.6
9,"27641 Deputy Cir, Laguna Hills, CA 92653","https://www.zillow.com/homes/27641-Deputy-Cir,...",3995000,6742,9.2
10,"27773 Hidden Trail Rd, Laguna Hills, CA 92653",https://www.zillow.com/homedetails/27773-Hidde...,2199000,4000,8.5
12,"8 Panorama, Coto De Caza, CA 92679",https://www.zillow.com/homedetails/8-Panorama-...,2899000,5000,8.4
13,"12 Vista Montemar, Laguna Niguel, CA 92677",https://www.zillow.com/homedetails/12-Vista-Mo...,3888000,5300,8.4
14,"25731 Bucklestone Dr, Laguna Hills, CA 92653",https://www.zillow.com/homedetails/25731-Buckl...,2500000,6799,8.3
15,"23 Via Lucena, San Clemente, CA 92673",https://www.zillow.com/homedetails/23-Via-Luce...,2275000,3786,8.0
17,"3 Canada Oaks, Coto De Caza, CA 92679",https://www.zillow.com/homedetails/3-Canada-Oa...,2999000,4400,7.8
